<a href="https://colab.research.google.com/github/Jk20873/hello/blob/main/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook you'll create your own bootstrap function following the bootstrap algorithm (check the lecture notes!)

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Load the data
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab2%20-%20bootstrap/customers.csv')
data = df.values.T[1]

In [ ]:
# Checking the notes from the lecture, create here your own bootstrap function:
# 1. Create an array of samples of shape (n_iterations, sample_size)
# 2. Calculate and save the mean of the array (this is "data_mean" that is returned by the function)
# 3. In each iteration:
# 3.1 Get the data corresponding to that iteration (new_samples[iteration, :])
# 3.2 Calculate the mean of the iteration data and store it
# (At this point you should have an array of n_iterations values)
# 4. Calculate the lower and upper bounds for a 95% CI (hint: check the percentile function on Numpy)
# 5. Return data_mean, and the lower and upper bounds of your interval
def bootstrap_mean(sample, sample_size, n_iterations):
  samples = np.zeros((n_iterations, sample_size))
  for i in range(n_iterations):
    samples[i, :] = np.random.choice(sample, (sample_size), replace=True)
  data_mean = np.mean(samples)
  samples_means = np.mean(samples, axis=1)
  lower = np.percentile(samples_means, 2.5)
  upper = np.percentile(samples_means, 97.5)
  return data_mean, lower, upper


In [ ]:
# Call your bootstrap function and plot the results

boots = []
for i in range(100, 50000, 1000):
    boot = bootstrap_mean(data, data.shape[0], i)
    boots.append([i, boot[0], "mean"])
    boots.append([i, boot[1], "lower"])
    boots.append([i, boot[2], "upper"])

df_boot = pd.DataFrame(boots, columns=['Bootstrap Iterations', 'Mean', "Value"])
sns_plot = sns.lmplot(df_boot.columns[0], df_boot.columns[1], data=df_boot, fit_reg=False, hue="Value")

sns_plot.axes[0, 0].set_ylim(0,)
sns_plot.axes[0, 0].set_xlim(0, 50000)
#sns_plot.savefig("bootstrap_confidence.png", bbox_inches='tight')
#sns_plot.savefig("bootstrap_confidence.pdf", bbox_inches='tight')


/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


(0.0, 50000.0)


Now, modify the bootstrap function you created above so that you can pass your desired confidence interval as a parameter.



In [ ]:
def bootstrap_mean_ci(sample, sample_size, n_iterations, ci):
  samples = np.zeros((n_iterations, sample_size))
  for i in range(n_iterations):
    samples[i, :] = np.random.choice(sample, (sample_size), replace=True)
  data_mean = np.mean(samples)
  shift = (100 - ci) / 2.0
  samples_means = np.mean(samples, axis=1)
  lower = np.percentile(samples_means, shift)
  upper = np.percentile(samples_means, 100 - shift)
  return data_mean, lower, upper

In [ ]:
boots = []
for i in range(100, 50000, 1000):
    boot = bootstrap_mean_ci(data, data.shape[0], i, 80)
    boots.append([i, boot[0], "mean"])
    boots.append([i, boot[1], "lower"])
    boots.append([i, boot[2], "upper"])

df_boot = pd.DataFrame(boots, columns=['Boostrap Iterations', 'Mean', "Value"])
sns_plot = sns.lmplot(df_boot.columns[0], df_boot.columns[1], data=df_boot, fit_reg=False, hue="Value")

sns_plot.axes[0, 0].set_ylim(0,)
sns_plot.axes[0, 0].set_xlim(0, 50000)

#sns_plot.savefig("bootstrap_confidence_80.pdf", bbox_inches='tight')


/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


(0.0, 50000.0)

# Vehicles dataset

Now let's work on a different dataset, which is stored in the vehicles.csv file.


In [ ]:
# Load and visualise the vehicles dataset
# To load the dataset: https://neptune.ai/blog/google-colab-dealing-with-files (check section "Load individual files directly from GitHub")


# Note that the current and new fleets are in different columns and have different lengths, so bear this in mind when you're plotting.
# You can create separate scatterplots for the two fleets, as you would with the histograms, 
# or plot them both in one plot (but not one against the other).
# <---INSERT YOUR CODE HERE--->
# Note: you can add more cells as needed to organise your code and your plots


## Compare the two fleets

The business analysts come up a comparison algorithm that requires the upper and lower bounds for the mean in order to say which fleet is better.
1. Calculate the mean of both samples.
2. Using the bootstrap function that you created:
    - Construct the 95% CI of the mean of the current fleet.
    - Construct the 95% CI of the mean of the new fleet.
    - Are they comparable? (i.e., is one better than the other?) -- you can do this with a permutation test (check the lecture notes!)

In [ ]:
# <---INSERT YOUR CODE HERE--->
# Load the data
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab2%20-%20bootstrap/vehicles.csv')
current_fleat = df['Current fleet']
new_fleat = df['New Fleet']
new_fleat = new_fleat[~np.isnan(new_fleat)]


In [ ]:
plt.scatter(range(len(current_fleat)), current_fleat, label='Current Fleet')
plt.scatter(range(len(new_fleat)), new_fleat, label='New Fleet')
plt.xlabel('Car ID')
plt.ylabel('MPG')
plt.title('MPG in Car Fleets')
plt.legend()

In [ ]:
sns.distplot(current_fleat, bins=20, kde=False, rug=True, axlabel='MPG').set_title('Histogram of Current Fleat')

In [ ]:
sns.distplot(new_fleat, bins=20, kde=False, rug=True, axlabel='MPG').set_title('Histogram of New Fleat')

In [ ]:
mean_current, lower_current, upper_current = bootstrap_mean_ci(current_fleat, current_fleat.shape[0], 100000, 95)
mean_new, lower_new, upper_new = bootstrap_mean_ci(new_fleat, new_fleat.shape[0], 100000, 95)

In [ ]:
print('Current Fleat:')
print('  Mean:', mean_current)
print('  Lower Mean (95% CI):', lower_current)
print('  Upper Mean (95% CI):', upper_current)
print()
print('New Fleat:')
print('  Mean:', mean_new)
print('  Lower Mean (95% CI):', lower_new)
print('  Upper Mean (95% CI):', upper_new)

Current Fleat:
  Mean: 20.143760883534135
  Lower Mean (95% CI): 19.353413654618475
  Upper Mean (95% CI): 20.943775100401606

New Fleat:
  Mean: 30.483100379746837
  Lower Mean (95% CI): 29.164556962025316
  Upper Mean (95% CI): 31.82278481012658


In [ ]:
def permutation_test_mean(arrA, arrB, n_permutations):
  concat = np.concatenate((arrA, arrB))
  arrAsize = len(arrA)
  meanA = np.mean(arrA)
  meanB = np.mean(arrB)
  meanDiffObs = meanB - meanA
  count = 0
  for i in range(n_permutations):
    perm = np.random.permutation(concat)
    permA = perm[:arrAsize]
    permB = perm[arrAsize:]
    meanPermA = np.mean(permA)
    meanPermB = np.mean(permB)
    if meanPermB - meanPermA > meanDiffObs:
      count = count + 1
  return count / n_permutations

In [ ]:
def permutation_test_mean(arrA, arrB, n_permutations):
  concat = np.concatenate((arrA, arrB))
  arrAsize = len(arrA)
  meanA = np.mean(arrA)
  meanB = np.mean(arrB)
  meanDiffObs = meanB - meanA
  count = 0
  for i in range(n_permutations):
    perm = np.random.permutation(concat)
    permA = perm[:arrAsize]
    permB = perm[arrAsize:]
    meanPermA = np.mean(permA)
    meanPermB = np.mean(permB)
    if meanPermB - meanPermA > meanDiffObs:
      count = count + 1
  return count / n_permutations